In [10]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
movie = pd.read_csv('/content/drive/MyDrive/movies/movies.csv')
rating = pd.read_csv('/content/drive/MyDrive/movies/ratings.csv')

movie.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [13]:
name_to_movieId = pd.Series(movie.movieId.values, index = movie.title).to_dict()

rating['liked'] = np.where(rating['rating']>=4, 1, 0) # df['rating']이 4이상면 재밌게 본 걸로
rating['movieId'] = rating['movieId'].astype('str')
gp_user_like = rating.groupby(['liked', 'userId'])
print(gp_user_like.groups)
movies_li = [gp_user_like.get_group(gp)['movieId'].to_list() for gp in gp_user_like.groups] # 한 row 당 한 사용자가 재밌게 본 영화id가 모여있음

{(0, 1): [1, 4, 6, 7, 10, 12, 13, 14, 15, 25, 27, 31, 32, 35, 40, 42, 45, 46, 47, 50, 51, 52, 53, 55, 59, 60, 61, 63, 65, 67, 68], (0, 2): [70, 71, 77, 78, 84, 86, 88, 91, 92, 94, 96, 99, 100, 105, 110, 116, 117, 122, 123, 124, 126, 129, 136, 139, 140, 141, 143, 145, 151, 153, 156, 163, 165, 167, 169, 171, 172, 174, 176, 177, 180, 184, 185, 186, 187, 189, 191, 195, 196, 198, 205, 209, 210, 214, 215, 216, 217, 221, 222, 225, 226, 228, 231, 233, 239, 244, 245, 246, 250, 251, 252], (0, 3): [260, 267, 268, 276, 282, 285, 298, 300, 305, 309, 314, 319, 325, 328, 329, 330, 332, 334, 337, 341, 343, 344, 345, 347, 348, 349, 350, 351, 352, 353, 355, 356, 358, 359, 360, 364, 366, 370, 373, 374, 375, 377, 378, 379, 382, 383, 384, 386, 387, 388, 391, 392, 394, 396, 397, 399, 402, 404, 406, 408, 410, 416, 418, 420, 421, 423, 424, 426, 427, 428, 429, 432, 433, 434, 435, 436, 437, 438, 440, 441, 442, 444, 445, 446, 447, 449, 451, 460, 461, 462, 463, 464, 466, 470, 472, 475, 476, 477, 478, 479, ...], (

In [14]:
model = Word2Vec(sentences = movies_li[:1000], # 1000개만 사용
                 min_count = 10, # 데이터 내에서 10번 이하로 등장한 영화는 제외
                 workers = 4, # 학습 시 쓰는 스레드 개수
                 sg = 1, # skip gram 사용
                 hs = 0, # negative sampling 사용
                 negative = 5, # If > 0, negative sampling 할때 뽑을 갯수
                 window = 9999999) # context window

model.save('movie_item2vec')

In [20]:
# model = Word2Vec.load('movie_item2vec')
movie_name = 'GoldenEye (1995)' # movies_li[:1000] 안에 영화가 있어야 함
recommended_li = model.wv.most_similar(str(name_to_movieId[movie_name]))
ids = []
for id, p in recommended_li:
  ids.append(int(id))

print('Recommendation on', movie_name)
movie.loc[movie['movieId'].isin(ids), :]

Recommendation on GoldenEye (1995)


,movieId,title,genres
151,153,Batman Forever (1995),Action|Adventure|Comedy|Crime
163,165,Die Hard: With a Vengeance (1995),Action|Crime|Thriller
183,185,"Net, The (1995)",Action|Crime|Thriller
206,208,Waterworld (1995),Action|Adventure|Sci-Fi
285,288,Natural Born Killers (1994),Action|Crime|Thriller
288,292,Outbreak (1995),Action|Drama|Sci-Fi|Thriller
324,329,Star Trek: Generations (1994),Adventure|Drama|Sci-Fi
344,349,Clear and Present Danger (1994),Action|Crime|Drama|Thriller
375,380,True Lies (1994),Action|Adventure|Comedy|Romance|Thriller
1341,1377,Batman Returns (1992),Action|Crime
